# E5 Using vector memory store to assemble coherent "what-if" scenarios

## 🔥 Get a kernel ready with a vector memory store

We're going to create a kernel that includes an embedding generation service — so that we can more easily absorb the knowledge that we're ✨ generating with the LLM for later reuse. BTW this is a common pattern we've seen with much larger businesses using Semantic Kernel today, but could apply at a smaller scale for the pizza shop owner:

1. Gather documents that are pertinent to the business in both structured and unstructured forms.
2. Create custom plugins to analyze and/or broaden the business' knowledge base for use by LLMs.
3. Move all the information into a vector database after converting it via embedding models.

Let's get more familiar with how embeddings work by ✨ generating more information that we can later work with.

In [13]:
from IPython.display import display, Markdown
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
    kernel.add_text_embedding_generation_service("ada", AzureTextEmbedding("text-embedding-ada-002", api_key, endpoint))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
    kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=ChromaMemoryStore(persist_directory='mymemories'))

Note that a new directory `mymemories` should be visible. If you want to start this set of lessons from the very beginning, delete the directory `mymemories` before you run the code above.

## 😶‍🌫️ Convert our SWOT into embeddings and store them in a memory collection.

We store the SWOT information in "conventional" arrays in Python that are not "semantic" in nature. That said, they're still eminently useful for the kind of computations we'll play with.

And to store the information semantically, we use the text embedding model added to the kernel to convert text into long vectors of numbers. We only need to do this once. But note that unless we have a "vector database" wired into the kernel, the embedding calculations do not last beyond this notebook. That's why it's called a "volatile" memory store.

In [14]:
strength_questions = ["What unique recipes or ingredients does the pizza shop use?","What are the skills and experience of the staff?","Does the pizza shop have a strong reputation in the local area?","Are there any unique features of the shop or its location that attract customers?", "Does the pizza shop have a strong reputation in the local area?", "Are there any unique features of the shop or its location that attract customers?"]
weakness_questions = ["What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)","Are there financial constraints that limit growth or improvements?","Are there any gaps in the product offering?","Are there customer complaints or negative reviews that need to be addressed?"]
opportunities_questions = ["Is there potential for new products or services (e.g., catering, delivery)?","Are there under-served customer segments or market areas?","Can new technologies or systems enhance the business operations?","Are there partnerships or local events that can be leveraged for marketing?"]
threats_questions = ["Who are the major competitors and what are they offering?","Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?","Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?","Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?"]

strengths = [ "Unique garlic pizza recipe that wins top awards","Owner trained in Sicily at some of the best pizzerias","Strong local reputation","Prime location on university campus" ]
weaknesses = [ "High staff turnover","Floods in the area damaged the seating areas that are in need of repair","Absence of popular calzones from menu","Negative reviews from younger demographic for lack of hip ingredients" ]
opportunities = [ "Untapped catering potential","Growing local tech startup community","Unexplored online presence and order capabilities","Upcoming annual food fair" ]
threats = [ "Competition from cheaper pizza businesses nearby","There's nearby street construction that will impact foot traffic","Rising cost of cheese will increase the cost of pizzas","No immediate local regulatory changes but it's election season" ]

print("✅ SWOT analysis for the pizza shop is resident in native memory")

memoryCollectionName = "SWOT"

for i in range(len(strengths)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"strength-{i}", text=f"Internal business strength (S in SWOT) that makes customers happy and satisfied Q&A: Q: {strength_questions[i]} A: {strengths[i]}")
for i in range(len(weaknesses)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"weakness-{i}", text=f"Internal business weakness (W in SWOT) that makes customers unhappy and dissatisfied Q&A: Q: {weakness_questions[i]} A: {weaknesses[i]}")
for i in range(len(opportunities)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"opportunity-{i}", text=f"External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: {opportunities_questions[i]} A: {opportunities[i]}")
for i in range(len(threats)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"threat-{i}", text=f"External threat (T in SWOT) to the business that impacts its survival Q&A: Q: {threats_questions[i]} A: {threats[i]}")

print("😶‍🌫️ Embeddings for SWOT have been generated")

✅ SWOT analysis for the pizza shop is resident in native memory
😶‍🌫️ Embeddings for SWOT have been generated


Keep in mind that weaknesses and threats are v similar but strengths and weaknesses usually pertain to internal factors; whereas opportunities and threats pertain to external factors. Play around with the `potential_question` variable below to get a sense of how the embedding model does with comparing your `potential_question` to the vectors that have been computed and stored in the step above.

In [15]:
potential_question = "what are easiest ways to make more money?"
counter = 0

memories = await kernel.memory.search_async(memoryCollectionName, potential_question, limit=5, min_relevance_score=0.5)

display(Markdown(f"### ❓ Potential question: {potential_question}"))

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    print(f"  > 😶‍🌫️ Similarity result {counter}:\n  >> ID: {memory.id}\n  Text: {memory.text}  Relevance: {memory.relevance}\n")

### ❓ Potential question: what are easiest ways to make more money?

  > 😶‍🌫️ Similarity result 1:
  >> ID: opportunity-0
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: Is there potential for new products or services (e.g., catering, delivery)? A: Untapped catering potential  Relevance: 0.7689356452897221

  > 😶‍🌫️ Similarity result 2:
  >> ID: opportunity-3
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: Are there partnerships or local events that can be leveraged for marketing? A: Upcoming annual food fair  Relevance: 0.7671651870349453

  > 😶‍🌫️ Similarity result 3:
  >> ID: opportunity-1
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: Are there under-served customer segments or market areas? A: Growing local tech startup community  Relevance: 0.7633657305250338

  > 😶‍🌫️ Similarity result 4:
  >> ID: opportunity-2
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&

The idea of having a semantic memory bank like this means a lot can be done with your LLM AI because you have a "vat" of knowledge available to you that can be sourced for processing through the LLM.

## Storing all the Business Thinking and Design Thinking into semantic memory

Let's take everything we have gathered thus far and put it into the semantic memory bank

In [16]:
pluginsDirectory = "./plugins-sk"

pluginBT = kernel.import_semantic_skill_from_directory(skillsDirectory, "BusinessThinking");

my_context = kernel.create_new_context()

the_business = 'makes pizzas'
my_context['input'] = the_business
my_context['strengths'] = ", ".join(strengths)
my_context['weaknesses'] = ", ".join(weaknesses)
my_context['opportunities'] = ", ".join(opportunities)
my_context['threats'] = ", ".join(threats)

print("✨ Computing cost efficiency...")
costefficiency_result = await kernel.run_async(pluginBT["SeekCostEfficiency"], input_context=my_context)
costefficiency_str = str("## Suggestions for how to gain cost efficiencies\n" + str(costefficiency_result))
await kernel.memory.save_information_async(memoryCollectionName, id="costefficiency", text=costefficiency_str)

print("✨ Computing time efficiency...")
timeefficiency_result = await kernel.run_async(pluginBT["SeekTimeEfficiency"], input_context=my_context)
timeefficiency_str = "## Suggestions for how to gain time efficiencies\n"+str(timeefficiency_result)
await kernel.memory.save_information_async(memoryCollectionName, id="timeefficiency", text=timeefficiency_str)

print("✨ Computing business strategies overview...")
bizstrat_result = await kernel.run_async(pluginBT["BasicStrategies"], input_context=my_context)
bizstrat_str = "# Business strategy thinking based on SWOT analysis\n"+str(bizstrat_result)
await kernel.memory.save_information_async(memoryCollectionName, id="bizstrat", text=bizstrat_str)

customer_comments = """
Customer 1: The seats look really raggedy.
Customer 2: The garlic pizza is the best on this earth.
Customer 3: I've noticed that there's a new server every time I visit, and they're clueless.
Customer 4: Why aren't there calzones?
Customer 5: I love the garlic pizza and can't get it anywhere else.
Customer 6: The garlic pizza is exceptional.
Customer 7: Pizza can get a little messy so I prefer a calzone's neatness.
Customer 8: Why is the pizza so expensive?
Customer 9: There's no way to do online ordering.
Customer 10: Why is the seating so uncomfortable and dirty?
"""

print("✨ Computing design thinking interim results from customer comments ...")
pluginDT = kernel.import_semantic_skill_from_directory(skillsDirectory, "DesignThinking");
my_result = await kernel.run_async(pluginDT["Empathize"], pluginDT["Define"], input_str=customer_comments)
designthinking_str = "# Categorized observations\n"+str(my_result)
await kernel.memory.save_information_async(memoryCollectionName, id="designthinking", text=designthinking_str)

print("😶‍🌫️ 🎉 Memory is now flush with information :+)")

✨ Computing cost efficiency...
✨ Computing time efficiency...
✨ Computing business strategies overview...
✨ Computing design thinking interim results from customer comments ...
😶‍🌫️ 🎉 Memory is now flush with information :+)


## 🧐 Put the accumulated memory to good use for "what if" scenarios

We can now put this information to work with the `FriendlyConsultant` plugin and have a special presentation generated on any "what if" scenario topic that we choose.

In [17]:
what_if_scenario = "How can the business owner save time?"
counter = 0

gathered_context = []
max_memories = 3
memories = await kernel.memory.search_async(memoryCollectionName, what_if_scenario, limit=max_memories, min_relevance_score=0.77)

print(f"✨ Leveraging information available to address '{what_if_scenario}'...")

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    gathered_context.append(memory.text + "\n")
    print(f"  > 😶‍🌫️ Hit {counter}: {memory.id} ")

skillsDirectory = "./plugins-sk"
print(f"✨ Synthesizing human-readable business-style presentation...")
pluginFC = kernel.import_semantic_skill_from_directory(skillsDirectory, "FriendlyConsultant");

my_context = kernel.create_new_context()
my_context['input'] = what_if_scenario
my_context['context'] = "\n".join(gathered_context)

preso_result = await kernel.run_async(pluginFC["Presentation"], input_context=my_context)

display(Markdown("# ✨ Generated presentation ...\n"+str(preso_result)))



✨ Leveraging information available to address 'How can the business owner save time?'...
  > 😶‍🌫️ Hit 1: timeefficiency 
  > 😶‍🌫️ Hit 2: costefficiency 
  > 😶‍🌫️ Hit 3: opportunity-2 
✨ Synthesizing human-readable business-style presentation...


# ✨ Generated presentation ...
---
# Business Strategy Consultant Presentation
## How to Save Time for the Business Owner

---
## The Question
How can the business owner save time?

---
## Three Key Concerns
1. Reduce workload and turnover rate
2. Free up time for experimentation with new menu items
3. Bridge the gap between traditional Sicilian style and desire for more modern, hip ingredients

---
## Concern 1: Reduce workload and turnover rate
**Explanation:** Implementing an online ordering system can reduce the need for staff to take orders, decreasing the workload and turnover rate.

**Example:** The business has a strong local reputation, but high staff turnover. By implementing an online ordering system, customers can place orders without the need for staff to take them, freeing up time for staff to focus on other tasks and reducing the workload. This can also lead to a decrease in turnover rate as staff may feel less overwhelmed and stressed.

---
## Concern 2: Free up time for experimentation with new menu items
**Explanation:** By streamlining kitchen processes, such as prepping ingredients and organizing workstations, the business can free up time to experiment with new menu items like calzones.

**Example:** The business has a unique garlic pizza recipe that wins top awards, but the absence of popular calzones from the menu may be limiting customer satisfaction and sales. By streamlining kitchen processes, staff can free up time to experiment with new menu items like calzones. This can lead to increased customer satisfaction and sales.

---
## Concern 3: Bridge the gap between traditional Sicilian style and desire for more modern, hip ingredients
**Explanation:** A comprehensive employee training program can help bridge the gap between the owner's traditional Sicilian style and the desire for more modern, hip ingredients, improving customer satisfaction and reducing negative reviews.

**Example:** The owner has been trained in Sicily at some of the best pizzerias, but negative reviews from younger demographics for lack of hip ingredients may be limiting customer satisfaction. By developing a comprehensive employee training program that emphasizes the unique garlic pizza recipe and provides opportunities for advancement, staff can learn how to bridge the gap between traditional Sicilian style and the desire for more modern, hip ingredients. This can lead to improved customer satisfaction and reduced negative reviews.

---
## Summary
To save time for the business owner, the consultant recommends implementing an online ordering system to reduce workload and turnover rate, streamlining kitchen processes to free up time for experimentation with new menu items, and developing a comprehensive employee training program to bridge the gap between traditional Sicilian style and the desire for more modern, hip ingredients. These suggestions can lead to increased customer satisfaction, sales, and a more efficient business operation. Additionally, exploring new technologies or systems such as an online presence and order capabilities can provide an external opportunity for the business to gain entirely new customers.

## 📘 A quick review before we head deeper into the pizza shop owner's challenges

1. Functions in SK enable encapsulating complicated AI computations in neat black box abstractons. 
2. Functions can be chained together with other functions and orchestrated serially by the kernel.  
3. Vector memory stores hold unstructured text data that is semantically searchable. 
4. Gathering relevant chunks of text improves a prompt's behavior — context has the power to guide.